In [1]:
import h5py # this might not be needed

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [24]:
import timeit, time

In [33]:
import pdb

In [1]:
import pandas as pd
import numpy as np
from keras import regularizers
from keras.models import Model
from keras.layers import Dense, Embedding, Input, LSTM, Bidirectional, GlobalMaxPool1D, Dropout
from keras.preprocessing import text, sequence
from keras.callbacks import EarlyStopping, ModelCheckpoint

/home/kai/anaconda3/envs/tf_gpu/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
PATH = '~/data/toxic/data/'

train = pd.read_csv(PATH + 'cleaned_train.csv')
test = pd.read_csv(PATH + 'cleaned_test.csv')

train_sentence = train['comment_text_cleaned']
test_sentence = test['comment_text_cleaned']

text_length = pd.concat([train_sentence.apply(lambda x: len(x.split())),\
                         test_sentence.apply(lambda x: len(x.split()))])

mean_length = text_length.mean()
std_length = text_length.std()

print(train.shape)
print(test.shape)
print(mean_length)
print(std_length)

(159571, 27)
(153164, 21)
72.20628647257263
114.01717258139988


In [59]:
# config
MAX_FEATURES = 100000 # max num of words
MAX_LEN = np.round(mean_length + 3*std_length).astype(int) # max sequence length
#EMBED_SIZE = 50 # embedding size
LSTM_UNITS = 50 # LSTM hidden layer unit number
DENSE_UNITS = 50
DROPOUT = 0.3 # dropout rate
BATCH_SIZE = 32
EPOCHS = 10


label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
print(MAX_LEN)

414


In [39]:
tokenizer = text.Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(pd.concat([train_sentence, test_sentence]).values)

In [40]:
train.comment_text_cleaned[0]

'explanation why the edit make under my username hardcore metallica fan be revert ? they be not vandalism just closure on some gas after i vote at new york doll fac . and please do not remove the template from the talk page since i be retire now .'

In [41]:
tokenized_train = tokenizer.texts_to_sequences(train_sentence.values)

In [47]:
len(tokenized_train), len(tokenized_train[0]), len(tokenized_train[1])

(159571, 46, 18)

In [48]:
print(tokenized_train[0])

[628, 79, 2, 33, 46, 197, 26, 702, 3682, 8108, 798, 1, 140, 45, 1, 12, 225, 50, 4961, 17, 64, 2014, 160, 5, 491, 31, 127, 1166, 8109, 2202, 7, 51, 14, 12, 98, 2, 278, 28, 2, 43, 23, 151, 5, 1, 2499, 90]


In [ ]:
tokenized_test = tokenizer.texts_to_sequences(test_sentence.values)

In [ ]:
X_train = sequence.pad_sequences(tokenized_train, maxlen=MAX_LEN)

In [ ]:
y = train[label_cols].values
X_test = sequence.pad_sequences(tokenized_test, maxlen=MAX_LEN)

In [34]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')

def get_embedding_matrix(embedding_file, embed_size, max_features, tokenizer):
    embeddings_index = dict(get_coefs(*o.strip().split()) for o in open(embedding_file, encoding='utf8'))
    all_embs = np.stack(embeddings_index.values())
    word_index = tokenizer.word_index
    nb_words = min(max_features, len(word_index))
    embedding_matrix = np.random.normal(all_embs.mean(), all_embs.std(), (nb_words, embed_size))
    for word, i in word_index.items():
        if i < max_features:
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    return embedding_matrix

def get_lstm_model(embedding_file, embed_size, max_features, tokenizer,\
                   max_len, lstm_units, dense_units, label_cols, dropout):
    embedding_matrix = get_embedding_matrix(embedding_file, embed_size, max_features, tokenizer)
    pdb.set_trace
    input = Input(shape=(max_len, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(input)
    x = Bidirectional(LSTM(lstm_units, return_sequences=True, dropout=0.2,\
                           kernel_regularizer=regularizers.l2(0.01)))(x)
    x = GlobalMaxPool1D()(x)
    x = Dropout(dropout)(x)
    x = Dense(dense_units, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
    x = Dense(len(label_cols), activation='sigmoid')(x)
    model = Model(inputs=input, outputs=x)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

def train_model(model, file_path, batch_size, epochs, X_train, y):
    checkpoint = ModelCheckpoint(file_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
    earlystopping = EarlyStopping(monitor="val_loss", mode="min", patience=20)
    callbacks_list = [checkpoint, earlystopping]
    model.fit(X_train, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)
    return model

def predict(model, file_path, X_test):
    model.load_weights(file_path)
    return model.predict(X_test, verbose=1)
    
def save(model_name, y_test, label_cols, path, is_train=False):
    if is_train:
        submission = pd.read_csv(path + 'sample_train.csv')
        file_name = 'train_' + model_name
    else:
        submission = pd.read_csv(path + 'sample_submission.csv')
        file_name = model_name # useless now
    submission[label_cols] = y_test
    print('submission shape:')
    print(submission.shape)
    submission.to_csv(path + 'sub_' + BUILD_ID + '.csv', index=False)
    
print('done')

done


In [32]:
X_train.shape

(159571, 414)

In [60]:
%%time

EMBEDDING_PATH = '/home/kai/data/resources/glove/'
for EMBED_SIZE in [100]:
    EMBEDDING_FILE_NAME = 'glove.6B.{}d.txt'.format(EMBED_SIZE)
    EMBEDDING_FILE = EMBEDDING_PATH + EMBEDDING_FILE_NAME
    print(EMBEDDING_FILE)
        
    start_time = timeit.default_timer()

    BUILD_ID = '{}_{}_{}_{}_{}_{}_{}_{}_{}'.format(MAX_FEATURES, EMBED_SIZE, LSTM_UNITS, DENSE_UNITS, DROPOUT, BATCH_SIZE, EPOCHS, EMBEDDING_FILE_NAME, int(time.time()))
    print('BUILD_ID: ' + BUILD_ID)
    SAVE_DIR = '/home/kai/data/shiyi/toxic/'
    MODEL_FILE = SAVE_DIR + 'models/mod_' + BUILD_ID + '.hdf5'
    print(MODEL_FILE)
    SUB_DIR = SAVE_DIR + 'submissions/'
    print(SUB_DIR)

    print('getting model')
    model = get_lstm_model(EMBEDDING_FILE, EMBED_SIZE, MAX_FEATURES, tokenizer,\
                           MAX_LEN, LSTM_UNITS, DENSE_UNITS, label_cols, DROPOUT)
    print('training')
    model = train_model(model, MODEL_FILE, BATCH_SIZE, EPOCHS, X_train, y)

    elapsed_time = (timeit.default_timer() - start_time)/3600    
    print('training (hours) {} '.format(elapsed_time))
    with open('timefile.txt','a') as f:
        f.write('##################################################################\n')
        f.write('##################################################################\n')
        f.write('\nBUILD_ID: ' + BUILD_ID + '\ntraining: '+str(elapsed_time))


    ############################################################################
    ############################################################################
    print('predicting')
    start_time = timeit.default_timer()

    y_test = predict(model, MODEL_FILE, X_test)
    #print('train predicting')
    #y_train = predict(model, MODEL_PATH, X_train)

    elapsed_time = (timeit.default_timer() - start_time)/3600    
    print('predicting (hours) {}'.format(elapsed_time))
    with open('timefile.txt','a') as f:
        f.write('\npredicting: '+str(elapsed_time)+'\n')

    save('lstm', y_test, label_cols, SUB_DIR)
    #save('lstm', y_train, label_cols, SUB_DIR, True)
    
    with open('timefile.txt','a') as f:
        model.summary(print_fn=lambda x: f.write(x + '\n'))
        for key, value in model.history.history.items():
            f.write('\nperformance: '+str(key)+': '+str(value))
        for key, value in model.history.params.items():
            f.write('\nparams: '+str(key)+': '+str(value))
        f.write('\n')
    
    print('done')

/home/kai/data/resources/glove/glove.6B.100d.txt
BUILD_ID: 100000_100_50_50_0.3_32_10_glove.6B.100d.txt_1518278460
/home/kai/data/shiyi/toxic/models/mod_100000_100_50_50_0.3_32_10_glove.6B.100d.txt_1518278460.hdf5
/home/kai/data/shiyi/toxic/submissions/
getting model
> <ipython-input-34-c7b3aa10e52b>(18)get_lstm_model()
-> input = Input(shape=(max_len, ))
(Pdb) c
training
Train on 143613 samples, validate on 15958 samples
Epoch 1/10
143584/143613 [============================>.] - ETA: 0s - loss: 0.0955 - acc: 0.9784
Epoch 00001: val_loss improved from inf to 0.05333, saving model to /home/kai/data/shiyi/toxic/models/mod_100000_100_50_50_0.3_32_10_glove.6B.100d.txt_1518278460.hdf5
143613/143613 [==============================] - 3870s 27ms/step - loss: 0.0955 - acc: 0.9784 - val_loss: 0.0533 - val_acc: 0.9820
Epoch 2/10
143584/143613 [============================>.] - ETA: 0s - loss: 0.0499 - acc: 0.9825
Epoch 00002: val_loss improved from 0.05333 to 0.05110, saving model to /home/kai/